
# Hierarchical Bayes (HB) — Beta‑Binomial Partial Pooling — Python Notebook

**When to Use**  
- You need **entity‑level estimates** (stores, geos, audiences, creatives) but each entity has **limited data**.  
- Want to **borrow strength** across entities while preserving heterogeneity.  
- Estimating **conversion rates**, **response probabilities**, or **success proportions** with uncertainty.

**Best Application**  
- Funnel metrics (CTR, CVR) at geo/store/audience level.  
- Small‑sample segments where stand‑alone MLEs are unstable.  
- Prioritization decisions (which geos to scale) with **posterior intervals**.

**When Not to Use**  
- When simple pooled model is sufficient (homogeneous entities) or when you need **strict frequentist inference** only.  
- If the goal is **causal effect estimation** under selection—use causal inference (IV/PSM/RCT) instead.

**How to Interpret Results**  
- **Posterior means** for each entity are **shrunk** toward the global mean; the amount of shrinkage depends on sample size and variance.  
- **Hyperparameters (α, β)** reflect the global prior over entity‑level probabilities.  
- Use **credible intervals** to compare entities and to make **risk‑aware** decisions.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.special import gammaln
from dataclasses import dataclass

pd.set_option('display.max_columns', 120)
plt.rcParams['figure.figsize'] = (8,4)
rng = np.random.default_rng(123)


### Data: Synthetic store‑level conversions (Beta‑Binomial ground truth)

In [ ]:

n_stores = 40
# True global prior for store conversion rates
alpha_true, beta_true = 2.5, 8.0
theta_true = rng.beta(alpha_true, beta_true, size=n_stores)

trials = rng.integers(60, 160, size=n_stores)      # impressions per store (toy)
conversions = rng.binomial(trials, theta_true)

df = pd.DataFrame({
    'store_id': np.arange(n_stores),
    'impressions': trials,
    'conversions': conversions,
    'ctr_empirical': conversions / trials
})

df.head()


### Model: Beta‑Binomial with hyperpriors and MCMC (Metropolis‑within‑Gibbs)

In [ ]:

# Likelihood:
# k_i | theta_i ~ Binomial(n_i, theta_i)
# theta_i | alpha, beta ~ Beta(alpha, beta)
# Hyperpriors: alpha ~ Gamma(a0, b0), beta ~ Gamma(a0, b0)  (shape-rate)
a0, b0 = 0.5, 0.5

def log_beta(a, b):
    return gammaln(a) + gammaln(b) - gammaln(a+b)

def log_p_alpha_beta(alpha, beta, k, n):
    # log posterior up to constant: sum_i log Beta(alpha+k_i, beta+n_i-k_i) - log Beta(alpha,beta)
    # + log Gamma prior
    if alpha <= 0 or beta <= 0:
        return -np.inf
    term = np.sum(log_beta(alpha + k, beta + n - k)) - n.size * log_beta(alpha, beta)
    # Gamma(a0,b0): log prior = (a0-1)log(x) - b0 x + const
    term += (a0-1)*np.log(alpha) - b0*alpha + (a0-1)*np.log(beta) - b0*beta
    return float(term)

@dataclass
class HBState:
    alpha: float
    beta: float
    theta: np.ndarray

def sample_thetas(alpha, beta, k, n, rng):
    # theta_i | rest ~ Beta(alpha+k_i, beta + n_i - k_i) (conjugate)
    return rng.beta(alpha + k, beta + n - k)

def mh_step_alpha_beta(state, k, n, step=0.25, rng=np.random.default_rng()):
    # Random walk on log scale for alpha and beta
    a_cur, b_cur = state.alpha, state.beta
    la_cur, lb_cur = np.log(a_cur), np.log(b_cur)
    la_prop = la_cur + rng.normal(0, step)
    lb_prop = lb_cur + rng.normal(0, step)
    a_prop, b_prop = np.exp(la_prop), np.exp(lb_prop)

    lp_cur = log_p_alpha_beta(a_cur, b_cur, k, n)
    lp_prop = log_p_alpha_beta(a_prop, b_prop, k, n)

    # Jacobian terms for log-transform proposals cancel in ratio if symmetric; accept/reject
    if np.log(rng.random()) < (lp_prop - lp_cur):
        return a_prop, b_prop, True
    else:
        return a_cur, b_cur, False

def run_mcmc(k, n, iters=4000, burn=1500, step=0.25, seed=123):
    rng = np.random.default_rng(seed)
    # Initialize with method-of-moments
    p_hat = k.sum() / n.sum()
    v_hat = np.var(k/n)
    # Rough init for alpha,beta
    s = 50 * p_hat * (1-p_hat) / max(v_hat, 1e-3)
    alpha = max(0.1, p_hat * s)
    beta = max(0.1, (1-p_hat) * s)
    theta = sample_thetas(alpha, beta, k, n, rng)

    trace_alpha, trace_beta = [], []
    accept = 0
    for t in range(iters):
        # Update thetas (conjugate)
        theta = sample_thetas(alpha, beta, k, n, rng)
        # Update alpha,beta via MH
        alpha, beta, acc = mh_step_alpha_beta(HBState(alpha, beta, theta), k, n, step=step, rng=rng)
        accept += int(acc)
        trace_alpha.append(alpha)
        trace_beta.append(beta)
    trace = pd.DataFrame({'alpha': trace_alpha, 'beta': trace_beta})
    post = trace.iloc[burn:].reset_index(drop=True)
    return post, theta, accept/iters

k = df['conversions'].values
n = df['impressions'].values
post, theta_last, acc_rate = run_mcmc(k, n, iters=5000, burn=2000, step=0.20, seed=9)
acc_rate


### Posterior for Hyperparameters α, β

In [ ]:

post_summary = post[['alpha','beta']].agg(['mean','std','median','quantile'])
post_summary.loc['q05'] = post[['alpha','beta']].quantile(0.05)
post_summary.loc['q95'] = post[['alpha','beta']].quantile(0.95)
post_summary


In [ ]:

fig, ax = plt.subplots()
ax.hist(post['alpha'], bins=40, alpha=0.6, label='alpha')
ax.hist(post['beta'], bins=40, alpha=0.6, label='beta')
ax.set_title('Posterior of alpha and beta')
ax.legend(); plt.show()


### Entity‑level Posteriors and Shrinkage vs. Raw Rates

In [ ]:

# Posterior mean for each theta_i under Beta(alpha+ki, beta+ni-ki) averaged over posterior draws of alpha,beta
# For speed, approximate using posterior means of alpha,beta
alpha_hat = post['alpha'].mean()
beta_hat  = post['beta'].mean()

df['theta_posterior_mean'] = (df['conversions'] + alpha_hat) / (df['impressions'] + alpha_hat + beta_hat)

# Compare to empirical CTR
plt.scatter(df['impressions'], df['ctr_empirical'], label='Empirical (MLE)', alpha=0.7)
plt.scatter(df['impressions'], df['theta_posterior_mean'], label='HB Posterior Mean', alpha=0.7)
plt.axhline((alpha_hat)/(alpha_hat+beta_hat), color='k', linestyle='--', label='Global mean')
plt.xlabel('Impressions (sample size)')
plt.ylabel('Rate')
plt.title('Shrinkage: HB vs Empirical Rates')
plt.legend(); plt.show()

df[['store_id','impressions','conversions','ctr_empirical','theta_posterior_mean']].head(10)


### Ranking Stores with Uncertainty (Posterior Intervals)

In [ ]:

# Compute posterior intervals by sampling theta_i conditional on alpha_hat, beta_hat (approximation)
S = 2000
theta_samples = np.zeros((len(df), S))
for s in range(S):
    theta_samples[:, s] = rng.beta(alpha_hat + df['conversions'].values,
                                   beta_hat + df['impressions'].values - df['conversions'].values)
low = np.quantile(theta_samples, 0.05, axis=1)
high = np.quantile(theta_samples, 0.95, axis=1)
df['theta_low95'] = low
df['theta_high95'] = high

top = df.sort_values('theta_posterior_mean', ascending=False).head(10)
top[['store_id','theta_posterior_mean','theta_low95','theta_high95']].round(4)


In [ ]:

# Visualize top 10 with intervals
tmp = top.sort_values('theta_posterior_mean', ascending=True)
plt.errorbar(tmp['theta_posterior_mean'], tmp['store_id'], 
             xerr=[tmp['theta_posterior_mean']-tmp['theta_low95'], 
                   tmp['theta_high95']-tmp['theta_posterior_mean']], 
             fmt='o')
plt.xlabel('Posterior rate (95% CI)')
plt.ylabel('Store ID')
plt.title('Top Stores by Posterior Rate with 95% CI')
plt.show()


### Posterior Predictive Check

In [ ]:

# Split stores into train/test by masking some stores (toy PPC)
mask = rng.random(len(df)) < 0.7
train_df = df[mask].copy()
test_df  = df[~mask].copy()

k_tr, n_tr = train_df['conversions'].values, train_df['impressions'].values
post_tr, _, _ = run_mcmc(k_tr, n_tr, iters=3000, burn=1500, step=0.2, seed=11)
a_hat, b_hat = post_tr['alpha'].mean(), post_tr['beta'].mean()

# Predictive distribution for held-out stores' conversions given impressions
S = 1000
pred_means, obs_rates = [], []
for _, row in test_df.iterrows():
    n_i = int(row['impressions'])
    # sample theta ~ Beta(a_hat,b_hat), then k ~ Binomial(n, theta)
    thetas = rng.beta(a_hat, b_hat, size=S)
    ks = rng.binomial(n_i, thetas)
    pred_means.append(ks.mean()/n_i)
    obs_rates.append(row['conversions']/n_i)

ppc_df = pd.DataFrame({'obs_rate': obs_rates, 'pred_mean_rate': pred_means})
ppc_df.head()


In [ ]:

plt.scatter(ppc_df['obs_rate'], ppc_df['pred_mean_rate'])
plt.plot([0,1],[0,1],'k--')
plt.xlabel('Observed Rate (Test)')
plt.ylabel('Posterior Predictive Mean Rate')
plt.title('Posterior Predictive Calibration (Held-out Stores)')
plt.show()



---

### Practical Guidance
- HB provides **robust small‑sample estimates** by shrinking extreme MLEs toward a global prior.  
- Use **entity‑level intervals** for risk‑aware ranking and to avoid over‑reacting to noise.  
- Extend to **covariates** by modeling logit(theta_i) with a hierarchical regression (requires MCMC/VI framework).

### References (non‑link citations)
1. Rossi, Allenby & McCulloch — *Bayesian Statistics and Marketing*.  
2. Gelman et al. — *Bayesian Data Analysis*.  
3. Fader & Hardie — *Probability Models for Customer‑Base Analysis*.
